In [1]:
print("Hello")

Hello


In [2]:
import mysql.connector
from mysql.connector import Error

hostname = "bx-7c.h.filess.io"
database = "olistproject_leatheron"
port = "3307"
username = "olistproject_leatheron"
password = "ded05b7b40d9bc990276c5528a90f307e0a5df35"

try:
    connection = mysql.connector.connect(host=hostname, database=database, user=username, password=password, port=port)
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")



Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "d:\Brazillian_Ecommerce_Bigdata_Project\venv\Lib\site-packages\IPython\core\interactiveshell.py", line 3579, in run_code
  File "C:\Users\Devthilina\AppData\Local\Temp\ipykernel_7232\3720461806.py", line 1, in <module>
    import mysql.connector
ModuleNotFoundError: No module named 'mysql'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "d:\Brazillian_Ecommerce_Bigdata_Project\venv\Lib\site-packages\pygments\styles\__init__.py", line 45, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "d:\Brazillian_Ecommerce_Bigdata_Project\venv\Lib\site-packages\IPython\core\interactiveshell.py", line 2170, in showtraceback
  File "d:\Brazillian_Ecommerce_Bigdata_Project\venv\Lib\site-packages\IPython\core\ultratb.py", line 1457, in structured_traceb